In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
import porespy as ps
import random

def load_raw_data(file_path, shape=(1000, 1000, 1000), dtype=np.uint8):
    with open(file_path, "rb") as f:
        raw_data = np.frombuffer(f.read(), dtype=dtype) 
    return raw_data.reshape(shape)  

def split_volume(volume, patch_size=500, stride=100):
    D, H, W = volume.shape
    def get_start_indices(size):
        indices = list(range(0, size - patch_size + 1, stride))
        if indices:
            last = indices[-1]
            if last + patch_size < size:
                indices.append(size - patch_size)
        print(indices)
        return indices

    d_starts = get_start_indices(D)
    h_starts = get_start_indices(H)
    w_starts = get_start_indices(W)

    sample = 0
    porosity_records = []

    for d in d_starts:
        for h in h_starts:
            for w in w_starts:
                sample += 1
                patch = volume[d:d + patch_size, h:h + patch_size, w:w + patch_size]
                save_path = os.path.join(output_dir, file, f"patch_{sample}.npy")
                np.save(save_path, patch)
                porosity = np.sum(patch == 0) / patch.size

                print(f"Saved patch to: {save_path}")
                print(f"Porosity: {porosity}")

                porosity_records.append({
                    "file": file,
                    "patch_index": sample,
                    "porosity": porosity
                })

    df = pd.DataFrame(porosity_records)
    csv_output_path = os.path.join(output_dir, file, "Porosity_segmented_rocks.csv")
    df.to_csv(csv_output_path, index=False)

raw_data_dir = "./rock"
output_dir = os.path.join(raw_data_dir, "patches_stride100")
os.makedirs(output_dir, exist_ok=True)
files = sorted([f for f in os.listdir(raw_data_dir) if f.endswith(".raw")],key=lambda x: int(x.split('_')[0]))

porosity_records = []
for file in files:
    file_path = os.path.join(raw_data_dir, file)
    volume = load_raw_data(file_path, shape=(1000, 1000, 1000), dtype=np.uint8)
    split_volume(volume, patch_size = 500, stride = 100)

In [ ]:
def generate_face_labels_with_features(array_3d):
    face_slices = {
        '-D': array_3d[0, :, :],
        '+D': array_3d[-1, :, :],
        '-W': array_3d[:, :, 0],
        '+W': array_3d[:, :, -1],
        '-H': array_3d[:, 0, :],
        '+H': array_3d[:, -1, :]
    }

    face_ids = {
        '-D':  1,
        '+D':  2,
        '-W':  3,
        '+W':  4,
        '-H':  5,
        '+H':  6
    }

    combinations = [
        ('-D', '-W', '-H'),
        ('-D', '+W', '-H'),
        ('-D', '-W', '+H'),
        ('-D', '+W', '+H'),
        ('+D', '-W', '-H'),
        ('+D', '+W', '-H'),
        ('+D', '-W', '+H'),
        ('+D', '+W', '+H')
    ]

    all_labels = []
    for combination in combinations:
        labels = []
        for face in combination:
            original_face = face_slices[face] 

            # SSA and Porosity Calculation
            mesh = ps.tools.mesh_region(region=original_face)
            surface_area = ps.metrics.mesh_surface_area(mesh=mesh)
            surface_area = np.log10(surface_area) 
            
            porosity = ps.metrics.porosity(1 - original_face)
            print(f"Face: {face}, Surface Area: {surface_area}, Porosity: {porosity}")

            surface_area_matrix = np.full_like(original_face, surface_area, dtype=np.float32)
            porosity_matrix = np.full_like(original_face, porosity, dtype=np.float32)
            
            face_id_value  = face_ids[face]
            face_id_matrix = np.full_like(original_face, face_id_value, dtype=np.float32)

            combined_face = np.stack([
                original_face,       
                porosity_matrix,     
                surface_area_matrix, 
                face_id_matrix       
            ], axis=0)

            labels.append(combined_face)
        all_labels.append(np.concatenate(labels, axis=0))  # (12, H, W)

    return np.array(all_labels, dtype=np.float32)  # (8, 12, H, W)    

def coarse_grid(array_3d, block_size=5):
    depth, height, width = array_3d.shape
    coarse_depth = depth // block_size
    coarse_height = height // block_size
    coarse_width = width // block_size
    
    coarse_array = np.zeros((coarse_depth, coarse_height, coarse_width), dtype=np.float32)
    
    for d in range(coarse_depth):
        for h in range(coarse_height):
            for w in range(coarse_width):
                block = array_3d[
                    d*block_size:(d+1)*block_size,
                    h*block_size:(h+1)*block_size,
                    w*block_size:(w+1)*block_size
                ]
                # calculate porosity in the block
                coarse_array[d, h, w] = np.sum(block == 0) / (block_size**3)
    
    return coarse_array

splitted_patches_dir = "./rock_patches"
output_hdf5_path = "./rock_500^3_2_100^3.h5"

patch_files = [f for f in os.listdir(splitted_patches_dir) if f.endswith(".npy")]
random.shuffle(patch_files, random=random.seed(42))
print(patch_files)

sample_index = 0
with h5py.File(output_hdf5_path, "w") as hdf5_file:
    for patch_file in patch_files:
        file_path = os.path.join(splitted_patches_dir, patch_file)
        patch = np.load(file_path)
        print(f"Processing {patch_file}.")
        
        coarse_image = coarse_grid(patch)
        label_data_list = generate_face_labels_with_features(patch)

        for label_idx, label_data in enumerate(label_data_list):
            dataset_name = f"sample_{sample_index}_combination_{label_idx}"
            print(f"Saving {dataset_name} (patch file: {patch_file}")
            group = hdf5_file.create_group(dataset_name)
            group.create_dataset("data", data=coarse_image, compression="gzip", compression_opts=9)
            group.create_dataset("label", data=label_data, compression="gzip", compression_opts=9)
            sample_index += 1

print(f"All data with labels saved to: {output_hdf5_path}")